# Hugging Face Library 'Transformer'およびT5Tokenizerのダウンロード

参考(https://qiita.com/takubb/items/fd972f0ac3dba909c293)これを基に改造し、最新のGoogle Colaboratoryで動作するようにした

In [1]:
!export CUDA_LAUNCH_BLOCKING=1
# !pip install torch
!pip install torchvision
!pip install transformers
#!apt install swig
# Sentencepieceのインストール
!pip install sentencepiece
!pip install mecab-python3
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
import gzip
import shutil
import sqlite3
import random
from math import ceil
import re
import csv
import glob
import torchvision
import statistics
import numpy as np
import lightgbm
import pickle
import statistics
import MeCab
import re
import copy

import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

# PyTorchとGPU設定

In [3]:
#!pip install torch
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Data Augmentation kansuu

In [4]:
# synreplace - replace kasho kosuu
# randinsert - tasu kotoba no kazu
# randdelete - delete kakuritsu
# randswap - swap kaisuu

class synreplace(object):
    def __init__(self, num):
        self.num = num
        self.model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
    def __call__(self, textlist):
        # textlist: honbun no list
        textlen = torch.where(textlist == 3)[0][0]
        for n in range(self.num):
            # chikan shiro
            masked_idx = random.randint(2, textlen-1)
            textlist[masked_idx] = 6
            # convert to tensor
            token_tensor = torch.tensor(textlist)
            # get the top 10 predictions of the masked token
            self.model = self.model.eval()
            with torch.no_grad():
                outputs = self.model(torch.unsqueeze(token_tensor, 0))
                predictions = outputs[0][0, masked_idx].topk(1)
            for i, index_t in enumerate(predictions.indices):
                index = index_t.item()
            textlist[masked_idx] = index
        return textlist

class randinsert(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        for n in range(self.num):
            insword = textlist[random.randint(1,len(textlist)-1)]
            i = random.randint(1,len(textlist)-1)
#            print('len: ', len(textlist))
#            print(i)
            while textlist[i] == 3:
                i = random.randint(1,len(textlist)-1)
#                print(i)
            textlist = torch.cat([textlist[0:i], torch.tensor([insword]), textlist[i:-1]])
        return textlist

class randdelete(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
#        print(textlist.shape)
        for i in range(3,len(textlist)-1):
            if textlist[i] == 3:
                continue
            r = random.uniform(0, 1)
            if r < self.num:
#                textlist.pop(i)
                textlist = torch.cat([textlist[0:i], textlist[i+1:], torch.tensor([3])])
#                print(textlist)
        return textlist

class randswap(object):
    def __init__(self, num):
        self.num = num
    def __call__(self, textlist):
        counter = 0
        #rs_sents = np.zeros(len(textlist), dtype=object)
        for i in range(len(textlist)):
            while self.num > counter:
                box = 0
                random_idx_1 = random.randint(1, len(textlist)-1)
                while textlist[random_idx_1] == 3:
                    random_idx_1 = random.randint(0, len(textlist)-1)
                random_idx_2 = random.randint(1, len(textlist)-1)
                while random_idx_1 == random_idx_2 or textlist[random_idx_2] == 3:
                    random_idx_2 = random.randint(0, len(textlist)-1)
                    # print(random_idx_1, random_idx_2)
                box = textlist[random_idx_1]
                textlist[random_idx_1] = textlist[random_idx_2]
                textlist[random_idx_2] = box
                counter += 1
        return textlist

# Custom Tensor Dataset
https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset

# データセットの準備

ライブドアニュースコーパスをダウンロード

    ダウンロードしたファイルは圧縮（tar.gz形式）ファイル
    様々なジャンル（IT,スポーツ,家電,映画など）のWEBメディアごとにフォルダに記事がテキストファイルで保存されている
    
以下、ファイルを読み込んで、必要な部分を抽出

In [5]:
#urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")
# ダウンロードした圧縮ファイルのパスを設定
#tgz_fname = "ldcc-20140209.tar.gz" 
# 2つをニュースメディアのジャンルを選定
mydata = '/export/livedoor' 
#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 

def remove_brackets(inp):
    output = re.sub(u'[〃-〿]', '',(re.sub('＝|=|×|\(|\)|“|”|（|）|／|\[|\]| |　|…|・|\n|\t|/|＜|＞|@|＠', '', re.sub(u'[ℊ-⿻]', '', inp)))) #210A ~ 2FFF
    return output

"""
def read_url(f):
    url = next(f)
    return url[:-1]

def read_date(f):
    date = next(f)
    date = remove_brackets(date.encode().decode('utf-8'))
    return date[:-1]
"""

def read_title(f):
    next(f)
    next(f)
    title = next(f)
    title = remove_brackets(title.encode().decode('utf-8'))
    return title[:-1]

def read_para(f):
    p = ''
    while True:
        try:
            para = next(f)
            para = remove_brackets(para.encode().decode('utf-8'))
            p += para
        except StopIteration:
            break
    return p [:-1]

In [6]:
directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/it-life-hack']
target_genre = ["dokujo-tsushin", "it-life-hack"] 
#directory = ['/export/livedoor/dokujo-tsushin', '/export/livedoor/peachy']
#target_genre = ["dokujo-tsushin", "peachy"] 
zero_fnames = []
one_fnames = []

if os.path.exists(tsv_fname) == True:
    with open(tsv_fname, "r+") as f:
        f.truncate(0)

for i in range(2):
    for filename in os.listdir(directory[i]):
        if "LICENSE.txt" in filename:
            continue
        f = os.path.join(directory[i], filename)
        #if os.path.isfile(f):
        #    print(f)
        if target_genre[0] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[0], '0', title, para]
                    writer.writerow(row)
            continue
        if target_genre[1] in f and f.endswith(".txt"):
            with open(tsv_fname, "a") as wf:
                writer = csv.writer(wf, delimiter='\t')
                with open(f) as zf:
                    title = read_title(zf)
                    para = read_para(zf)
                    row = [target_genre[1], '1', title, para]
                    writer.writerow(row)
            continue

pandasでデータを読み込み

In [7]:
import pandas as pd
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label','title','sentence'])
# データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (1740, 4)


,media_name,label,title,sentence
548,dokujo-tsushin,0,夏バテよりも深刻な新型秋バテは、1日2回の紅茶でリセット!,あつい日がまだまだ続いていますが、みなさん、節電してますか？もとは私たちの配慮だったはずが、...
1327,it-life-hack,1,何だそうだったのか！Googleロゴがキノコになっている理,納得！そういうことか！Googleロゴにファスナーが付いてる理由で紹介したように、Googl...
200,dokujo-tsushin,0,婚期短し恋せよ乙女！乙女女子解体新,近年、大人女子、肉食女子、腐女子etcといったある種の女性たちのことを女子と総称する呼び名が...
154,dokujo-tsushin,0,間取り図にハマる女ゴコ,先日、今までよりも一部屋多いマンションに引っ越したマリさん29歳旅行関係。希望に合う物件を探...
191,dokujo-tsushin,0,ケンカをしても気持ちが冷めている。長すぎた春の予,先日、28歳の誕生日を迎えたジュンさんアパレルメーカー勤務。以前から欲しかった指輪を彼にプレ...
286,dokujo-tsushin,0,あなたがお局と呼ばれる理,化粧品メーカーの派遣社員の歩さん23歳の職場は50代の主任と20代後半の先輩スタッフの3人。...
1571,it-life-hack,1,無料PDF作成ソフトを大活用！PDFでビジネスが変わる知っ得！虎の,ふだん何気なく使っているパソコンだが、ちょっとした工夫やヒントで効率よく仕事ができたり、今ま...
1399,it-life-hack,1,英単語をイメージ化して覚える！全単語イラスト付きのワードスケッチTOEICテストiPhone...,英単語を覚えるのに、昔は出る単や豆単を使って必死で覚えたなんて記憶ありませんか？いまではネッ...
1017,it-life-hack,1,iPhone5用ケース続先行発売中！存在しないはずのiPhone5用ケースが通販,現在はMacRumorsMacのうわさというサイトでしか見ることができないが、6日ほど前にK...
1179,it-life-hack,1,売れ筋モデルがさらにコンパクトにEPSONより2012年インクジェット新モデル登,8月21日にエプソンがビジネス向けインクジェットプリンターの新モデルを発表したことはお伝えし...


//文章データをsentences、ラベルデータを labelsに保存、以降この2変数だけを利用

In [8]:
mn = df.media_name.values
labels = df.label.values
titles = df.title.values
sentences = df.sentence.values

In [9]:
print(type(labels))

<class 'numpy.ndarray'>


In [10]:
tagger = MeCab.Tagger("-Owakati")

def make_wakati(sentence):
  # MeCabで分かち書きを行う
    sentence = tagger.parse(sentence)
  # 半角全角英数字などは削除する
#    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
  # 記号なども削除する
#    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
  # スペース区切で形態素の配列に変換する
    wakati = sentence.split(" ")
  # 空要素を削除する
    wakati = list(filter(("").__ne__, wakati))
    return wakati

In [11]:
hoge=make_wakati(sentences[2])
hoge[0]

'結婚'

In [12]:
wakati_sentences = []

for i in range(len(sentences)):
    wakati_sentences.append(make_wakati(sentences[i]))

In [13]:
hoge = []
hoge.append(wakati_sentences[0][0])
hoge += wakati_sentences[0][1]
hoge += wakati_sentences[0][2]
hoge

['両親', 'や', '親', '族']

In [14]:
for i in enumerate(wakati_sentences):
    continue
print(i[0])

1739


In [15]:
wcount = 128

h_sentences = []
t_sentences = []

# wcount moji me kara kesu

for i in enumerate(wakati_sentences):
    #h_sent = []
    h_len = 0
    hn = 0
    #h_sent.append(wakati_sentences[i[0]][0])
    h_len += len(wakati_sentences[i[0]][0])
    while h_len < wcount:
        try:
            hn += 1
            if wakati_sentences[i[0]][hn]:
                #print(hn, wakati_sentences[i[0]][hn])
                #h_sent.append(wakati_sentences[i[0]][hn])
                h_len += len(wakati_sentences[i[0]][hn])
        except IndexError:
            break
    h_sentences.append(sentences[i[0]][:hn])
    
    #t_sent = []
    t_len = 0
    tn = 2
    #t_sent.append(wakati_sentences[i[0]][-2])
    t_len += len(wakati_sentences[i[0]][-2])
    while t_len < wcount:
        try:
            tn += 1
            if wakati_sentences[i[0]][tn]:
                #print(tn, wakati_sentences[i[0]][tn])
                #t_sent.append(wakati_sentences[i[0]][tn])
                t_len += len(wakati_sentences[i[0]][tn])
        except IndexError:
            break
    t_sentences.append(sentences[i[0]][-tn:])

In [16]:
h_sentences[0]

'両親や親族が高齢なってくると、自分の人生を見つめ直さないといけないことがある。自分が独女の場合はなお更である。今回の相談者ミミさん女性38歳は母親が介護が必要になった'

In [17]:
hsentences = np.array(h_sentences)
tsentences = np.array(t_sentences)

In [18]:
emp = []
asentences = np.append(emp, copy.deepcopy(sentences))
ksentences = np.append(emp, copy.deepcopy(sentences))
kksentences = np.append(emp, copy.deepcopy(sentences))

# wcount moji me kara kesu

for i in enumerate(sentences):
    if len(i[1])>wcount:
        asentences[i[0]] = sentences[i[0]][:wcount]

# ushiro kara wcount moji toru

for i in enumerate(sentences):
    if len(i[1])>wcount:
        ksentences[i[0]] = sentences[i[0]][-wcount:]

# ushiro kara wcount moji toru ichiban ketsu wa toranai

for i in enumerate(sentences):
    if len(i[1])>wcount:
        am = wcount+10
        a = sentences[i[0]][-am:]
        kksentences[i[0]] = a[:wcount]

# BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
単語分割とIDへ変換

# テスト実行

In [19]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in hsentences:
    # Tokenizeで分割
    h_token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(h_token_words))
for sent in tsentences:
    # Tokenizeで分割
    t_token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(t_token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

最大単語数:  63
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [20]:
h_input_ids = []
h_attention_masks = []

# 1文づつ処理
for sent in hsentences:
    hencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    h_input_ids.append(hencoded_dict['input_ids'])
    h_attention_masks.append(hencoded_dict['attention_mask'])
    

# リストに入ったtensorを縦方向（dim=0）へ結合
h_input_ids = torch.cat(h_input_ids, dim=0)
h_attention_masks = torch.cat(h_attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

# 確認
print('Original: ', hsentences[0])
print('Token IDs:', h_input_ids[0])
print(type(labels), labels)
print(torch.Tensor.size(h_input_ids))
print(torch.Tensor.size(h_attention_masks))
print(torch.Tensor.size(labels))
print(hsentences.size)

Original:  両親や親族が高齢なってくると、自分の人生を見つめ直さないといけないことがある。自分が独女の場合はなお更である。今回の相談者ミミさん女性38歳は母親が介護が必要になった
Token IDs: tensor([    9,  3652,    26, 13508,    12, 19860,    57,  9123,    20,     7,
         1393,  4221,  1429, 15133,   442,  4569,    20, 19133,  1272,     8,
         5144,  2596,   612,  2855,  3244, 13426,    27,     8, 16058,  5448,
          147,   318,   318,   774,   577,  1300,   559,    11,  4180,    12,
        12079,  3413,   344,     2,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3, 

In [21]:
h_input_ids

tensor([[    9,  3652,    26,  ...,     3,     3,     3],
        [    9,  3911,    10,  ...,     3,     3,     3],
        [    9,  1743, 26243,  ...,     3,     3,     3],
        ...,
        [    9,  1387,  1773,  ...,     3,     3,     3],
        [ 3502, 12393,    26,  ...,     3,     3,     3],
        [  673,   223,  5874,  ...,     3,     3,     3]])

In [22]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torchvision import transforms, datasets
from transformers import RobertaForMaskedLM

# データセットクラスの作成
# dataset = TensorDataset(h_input_ids, h_attention_masks, labels)
hdataset = TensorDataset(h_input_ids, h_attention_masks, labels)
# dataset = CustomTensorDataset(tensors = (input_ids, labels), transform = None)


# KOKO

In [23]:
# 80%地点のIDを取得
num_dataset = len(hdataset)
#train_size = int(0.1 * num_dataset)
#val_size = num_dataset - train_size

train_size = 50
val_size = len(hdataset) - train_size

# データセhttp://localhost:8888/notebooks/bert-zuco/augmentation/BERTclassification-mydata-local-augmentation-aug.ipynb#ットを分割
h_train_dataset, h_val_dataset = random_split(hdataset, [train_size, val_size])

print('訓練データ数:{}'.format(train_size))
print('検証データ数:{}'.format(val_size))

# データローダーの作成
batch_size = 40
# batch_size = 25

data_transform = transforms.Compose([
    synreplace(1),
    randinsert(3),
    randdelete(0.15), 
    randswap(2)
])


class MySubset(torch.utils.data.Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform

    def __getitem__(self, idx):
        xa, mask, label = self.dataset[self.indices[idx]]
        if self.transform:
            xa = self.transform(xa)
        return xa, mask, label

    def __len__(self):
        return len(self.indices)
    
indices = np.random.choice(num_dataset, num_dataset, replace=False)

h_train_dataset = MySubset(hdataset, indices[:train_size], data_transform)
h_val_dataset = MySubset(hdataset, indices[train_size:])

# 訓練データローダー
h_train_dataloader = DataLoader(
            h_train_dataset,  
            sampler = RandomSampler(h_train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
            )



# 検証データローダー
h_validation_dataloader = DataLoader(
            h_val_dataset, 
            sampler = SequentialSampler(h_val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )


訓練データ数:50
検証データ数:1690


In [24]:
len(h_train_dataset)

50

In [25]:
from transformers import BertForSequenceClassification,AdamW,BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinaryなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()


2022-11-07 02:13:21.145716: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 02:13:21.262736: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 02:13:21.607424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-07 02:13:21.607470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [26]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def h_train(model):
    model.train() # 訓練モードで実行
    h_train_loss = 0
    for batch in h_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        h_train_loss += loss.item()
    return h_train_loss


# テストパートの定義
def h_validation(h_model):
    model.eval()# 訓練モードをオフ
    h_val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in h_validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            h_val_loss += loss.item()
    return h_val_loss


In [27]:
# 学習の実行
max_epoch = 50
h_train_loss_ = []
h_test_loss_ = []

h_train_loss = 0

for epoch in range(max_epoch):
    h_train_ = h_train(model)
    h_test_ = h_train(model)
    h_train_loss_.append(h_train_)
    h_test_loss_.append(h_test_)

In [28]:
# b_input_mask.size(), b_input_ids.size(), labels.size()
# outputs = self.model(torch.unsqueeze(token_tensor, 0))

In [29]:
hoge = h_train_dataloader.__iter__()
hoge.next()

[tensor([[    9,     3,  8680,  ...,     3,     3,     3],
         [    9,   539,     3,  ...,     3,     3,     3],
         [    9,     0,  1075,  ...,     3,     3,     3],
         ...,
         [    9,   474,    17,  ...,     3,     3,     3],
         [    9,   539,     9,  ...,     3,     3,     3],
         [    9,     3, 11344,  ...,     3,     3,     3]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
         0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1])]

In [30]:
model.train() # 訓練モードで実行
h_test_accuracy = []

for epoch in range(max_epoch):
    h_train_ = h_train(model)
    h_test_ = h_train(model)
    h_train_loss_.append(h_train_)
    h_test_loss_.append(h_test_)

for batch in h_train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
#    b_input_ids = batch[0].to(device)
#    b_input_mask = batch[1].to(device)
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = torch.t(batch[2]).to(device)
    optimizer.zero_grad()
    outputs = model(b_input_ids, 
                         token_type_ids=None, 
                         attention_mask=b_input_mask, 
                         labels=b_labels)
    h_loss = outputs.loss
    h_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    h_train_loss += h_loss.item()

model.eval()# 訓練モードをオフ

for batch in h_validation_dataloader:
    #print(batch)
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    #print(len(b_labels))
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        h_preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        h_test_accuracy.append((torch.argmax(h_preds[0], 1) == b_labels).sum().item() / len(b_labels))


In [31]:
sents = []

for i in range(df.shape[0]):
    sents.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[i])))

#print(sents)
sents = pd.DataFrame(sents)
print(type(sents))


<class 'pandas.core.frame.DataFrame'>


In [34]:
lsentences = np.append(emp, copy.deepcopy(sentences))

l_input_ids = []
l_attention_masks = []

for sent in lsentences:
    aencoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = wcount+2,  # I think maximum 文章の長さを固定（Padding/Trancatinating）
                        truncation=True,                
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )
    # print(hencoded_dict)

    l_input_ids.append(aencoded_dict['input_ids'])
    l_attention_masks.append(aencoded_dict['attention_mask'])

l_input_ids = torch.cat(l_input_ids, dim=0)
l_attention_masks = torch.cat(l_attention_masks, dim=0)
ldataset = TensorDataset(l_input_ids, h_attention_masks, labels)
l_train_dataset, l_val_dataset = random_split(ldataset, [train_size, val_size])

In [36]:
l_train_dataset[1][2]

l_preds = []

for i in range(len(l_train_dataset)):
    l_preds.append(l_train_dataset[i][2])

l_preds = np.array(l_preds)

In [38]:
print(len(l_train_dataset))
print(len(h_preds[0]))

50
10


In [40]:
l_preds

array([0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 1])

# type soroete X train test Y train test wo kaizan suru

In [42]:
'''
X_train, X_test, y_train, y_test = train_test_split(
    sents, labels, test_size=0.95, random_state=42, shuffle=True
    )
print(y_train)

# x ga transformer id  y ga raberu
'''

'\nX_train, X_test, y_train, y_test = train_test_split(\n    sents, labels, test_size=0.95, random_state=42, shuffle=True\n    )\nprint(y_train)\n\n# x ga transformer id  y ga raberu\n'

In [44]:
'''
print(type(y_train))
print(type(l_train_dataset[0][2]))
'''

'\nprint(type(y_train))\nprint(type(l_train_dataset[0][2]))\n'

In [46]:
Xtrain = l_train_dataset[:][0] 
Xtest = l_val_dataset[:][0]
ytrain = l_train_dataset[:][2]
ytest = l_val_dataset[:][2]

X_train = Xtrain.to('cpu').detach().numpy().copy()
X_test = Xtest.to('cpu').detach().numpy().copy()
y_train = ytrain.to('cpu').detach().numpy().copy()
y_test = ytest.to('cpu').detach().numpy().copy()

In [48]:
lgb_clf = lightgbm.LGBMClassifier(max_depth=50, num_iterations=30)
eval_res = {}
lgb_clf.fit(X_train, y_train, eval_set=[(X_test, y_test), (X_train, y_train)], verbose=5)

[5]	training's binary_logloss: 0.59217	valid_0's binary_logloss: 0.704068
[10]	training's binary_logloss: 0.529546	valid_0's binary_logloss: 0.716894
[15]	training's binary_logloss: 0.483243	valid_0's binary_logloss: 0.726575
[20]	training's binary_logloss: 0.446107	valid_0's binary_logloss: 0.738829
[25]	training's binary_logloss: 0.415453	valid_0's binary_logloss: 0.751629
[30]	training's binary_logloss: 0.388572	valid_0's binary_logloss: 0.758783


LGBMClassifier(max_depth=50, num_iterations=30)

In [50]:
y_pred = lgb_clf.predict(X_test)
print(y_pred)
print(len(y_pred))

[1 0 1 ... 0 1 1]
1690


In [52]:
y_pred = lgb_clf.predict(X_test)
print('head: ', statistics.mean(h_test_accuracy))

head:  0.5587209302325582


In [54]:
len(h_preds[0].cpu().numpy())

10

In [56]:
h_pred_df = pd.DataFrame(np.argmax(h_preds[0].cpu().numpy(), axis=1), columns=['h_pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])
accuracy_df = pd.concat([h_pred_df, label_df], axis=1)
accuracy_df

,h_pred_label,true_label
0,0,1
1,0,1
2,0,1
3,0,0
4,0,1
5,0,0
6,0,0
7,0,0
8,0,1
9,0,0


In [58]:
a = torch.randn(4, 4)
print(a)
print(torch.argmax(a))

tensor([[ 0.4946,  1.6461,  0.7885,  1.0125],
        [-0.3847,  1.0288,  1.0468, -0.2360],
        [-0.4364,  0.5339,  0.1872,  0.2695],
        [ 1.0386, -0.1221,  0.0228, -2.0122]])
tensor(1)


In [61]:
# 予測結果の確認
print(f'出力:{h_preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[ 3.5504, -3.6462],
        [ 3.3683, -3.4001],
        [ 3.5343, -3.5912],
        [ 3.3439, -3.4602],
        [ 3.2887, -3.4893],
        [ 3.4014, -3.5296],
        [ 3.5517, -3.6730],
        [ 3.4991, -3.5611],
        [ 3.3458, -3.4185],
        [ 3.4956, -3.5425]], device='cuda:0'), hidden_states=None, attentions=None)


In [63]:
'''
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])
accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df.head()
'''

"\n# 比較しやすい様にpd.dataframeへ整形\nimport numpy as np\n# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）\nlogits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])\n## np.argmaxで大き方の値を取得\npred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])\nlabel_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])\naccuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)\naccuracy_df.head()\n"

In [65]:
'''
correct = 0
test_accuracy = 0

p_list = []
l_list = []

p_list = pred_df.values.tolist()
l_list = label_df.values.tolist()

#print(type(pred_df))

for i in range(len(b_labels)):
    if(p_list[i] == l_list[i]):
        correct += 1

test_accuracy = correct/len(b_labels)

print(test_accuracy)
'''

'\ncorrect = 0\ntest_accuracy = 0\n\np_list = []\nl_list = []\n\np_list = pred_df.values.tolist()\nl_list = label_df.values.tolist()\n\n#print(type(pred_df))\n\nfor i in range(len(b_labels)):\n    if(p_list[i] == l_list[i]):\n        correct += 1\n\ntest_accuracy = correct/len(b_labels)\n\nprint(test_accuracy)\n'

In [67]:
len(h_test_loss_)

100

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(h_test_loss_)), h_test_loss_, label="Head")
plt.plot(range(len(t_test_loss_)), t_test_loss_, label="Tail")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()  # ラベルがあるときは、きちんとplt.show()を呼び出すこと